<p>Here we will check if the subroutes we intend to use for modelling cover the stops on all other subroutes i.e. those that will be excluded. We will check for additional stops for that line in that direction that are not covered by the longest subroute.</p>

In [68]:
import pandas as pd
import sqlite3
import numpy as np

conn = sqlite3.connect("busdata_18.sqlite")
cur = conn.cursor()

In [69]:
# Query database
q = "SELECT * FROM trips_cleaned;"
trips_df = pd.read_sql_query(q, conn)

In [70]:
# trips_df.dropna(inplace=True)

In [71]:
# trips_df contains cleaned data from RT_TRIPS
trips_df.head()

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP
0,07-FEB-18 00:00:00,6253783,68,68_80,1,87245,84600,87524.0,84600.0
1,07-FEB-18 00:00:00,6262138,25B,25B_271,2,30517,26460,32752.0,NaN
2,07-FEB-18 00:00:00,6254942,45A,45A_70,2,35512,32100,36329.0,32082.0
3,07-FEB-18 00:00:00,6259460,25A,25A_273,1,57261,54420,58463.0,54443.0
4,07-FEB-18 00:00:00,6253175,14,14_15,1,85383,81600,84682.0,81608.0


In [92]:
trips_df.loc[trips_df["LINEID"] == "41X"]

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP
4568,26-SEP-18 00:00:00,7771899,41X,41X_128,2,32327,27900,33703.0,27903.0
6474,20-SEP-18 00:00:00,8095596,41X,41X_121,1,67775,63000,69170.0,63055.0
7711,22-JUN-18 00:00:00,7021056,41X,41X_128,2,32356,27900,32907.0,27909.0
8722,20-SEP-18 00:00:00,8097148,41X,41X_125,2,32159,28200,33794.0,28252.0
8723,20-SEP-18 00:00:00,8096131,41X,41X_125,2,32159,28200,NaN,28410.0
...,...,...,...,...,...,...,...,...,...
2179927,15-MAY-18 00:00:00,6753699,41X,41X_123,1,65940,61200,67377.0,61109.0
2181390,15-MAY-18 00:00:00,6758489,41X,41X_125,2,31373,27600,32205.0,27678.0
2181497,15-MAY-18 00:00:00,6763923,41X,41X_128,2,32251,27900,33734.0,27871.0
2181960,15-MAY-18 00:00:00,6756962,41X,41X_125,2,31973,28200,32440.0,NaN


In [72]:
# Create list of lines
all_lines = trips_df.LINEID.unique().tolist()
all_lines.sort()

In [73]:
print(all_lines)

['1', '102', '104', '11', '111', '114', '116', '118', '120', '122', '123', '13', '130', '14', '140', '142', '145', '14C', '15', '150', '151', '15A', '15B', '15D', '16', '161', '16C', '16D', '17', '17A', '18', '184', '185', '220', '236', '238', '239', '25', '25A', '25B', '25D', '25X', '26', '27', '270', '27A', '27B', '27X', '29A', '31', '31A', '31B', '31D', '32', '32X', '33', '33A', '33B', '33D', '33E', '33X', '37', '38', '38A', '38B', '38D', '39', '39A', '39X', '4', '40', '40B', '40D', '40E', '41', '41A', '41B', '41C', '41D', '41X', '42', '42D', '43', '44', '44B', '45A', '46A', '46E', '47', '49', '51D', '51X', '53', '54A', '56A', '59', '61', '63', '65', '65B', '66', '66A', '66B', '66X', '67', '67X', '68', '68A', '68X', '69', '69X', '7', '70', '70D', '75', '76', '76A', '77A', '77X', '79', '79A', '7A', '7B', '7D', '83', '83A', '84', '84A', '84X', '9']


In [74]:
longest_df = pd.read_csv("~/data/stoptimes/stops_per_main_subroutes.csv")

In [82]:
# longest_df contains stop numbers and sequence per longest subroute per line
longest_df.head()

,LINEID,ROUTEID,DIRECTION,PROGRNUMBER,STOPPOINTID
0,102,102_8,1,1,4381
1,102,102_8,1,2,935
2,102,102_8,1,3,936
3,102,102_8,1,4,913
4,102,102_8,1,5,914


In [83]:
# Create list of longest subroutes
longest_subroutes = longest_df.ROUTEID.unique().tolist()
longest_subroutes.sort()

In [84]:
print(longest_subroutes)

['102_8', '102_9', '104_15', '104_16', '111_7', '111_8', '111_9', '114_5', '114_6', '116_1', '116_3', '118_4', '11_40', '11_42', '120_10', '120_11', '120_8', '122_14', '122_16', '122_18', '122_20', '123_34', '123_36', '130_10', '130_11', '13_61', '13_63', '13_67', '13_68', '13_70', '140_19', '140_21', '142_12', '142_13', '145_102', '14C_17', '14C_18', '14_15', '14_16', '150_8', '150_9', '151_15', '151_16', '151_17', '15A_83', '15A_84', '15B_56', '15B_60', '15B_61', '15D_62', '15D_63', '15_16', '15_17', '161_50', '161_51', '16C_28', '16C_29', '16D_30', '16_20', '16_24', '17A_11', '17A_13', '17A_15', '17A_17', '17_10', '17_15', '184_28', '184_29', '185_54', '185_55', '185_59', '185_60', '185_62', '18_3', '18_4', '1_37', '1_40', '220_15', '236_10', '236_9', '238_11', '238_15', '239_26', '239_28', '25A_270', '25A_273', '25B_271', '25B_274', '25D_275', '25D_277', '25X_10', '25X_9', '25_269', '25_272', '26_24', '26_26', '26_28', '26_29', '270_42', '270_44', '27A_4', '27A_7', '27B_23', '27B_3

In [91]:
problematic_routes = []

for route in longest_subroutes:
    line = route.split("_")[0]
    
    longest_subroute_df = longest_df.loc[longest_df["ROUTEID"] == route]
    subroute_direction = longest_subroute_df.iloc[0]["DIRECTION"]
    stops_per_subroute = longest_subroute_df.STOPPOINTID.unique().tolist()
    stops_per_subroute.sort()
    
    line_df = pd.read_csv("~/data/stoptimes/" + line + "_stops.csv")
    merged = line_df.merge(trips_df, how="left", on=["DAYOFSERVICE", "TRIPID"])
    line_subset = merged.loc[merged["DIRECTION"] == subroute_direction]
    stops_per_line = line_subset.STOPPOINTID.unique().tolist()
    stops_per_line.sort()
    
    if stops_per_line == stops_per_subroute:
        print("Subroute", route, "ok")
    else:
        problematic_routes.append(route)
        print("Stops excluded for subroute", route +":", np.setdiff1d(stops_per_line, stops_per_subroute))
    

Subroute 102_8 ok
Subroute 102_9 ok
Subroute 104_15 ok
Subroute 104_16 ok
Subroute 111_7 ok
Stops excluded for subroute 111_8: [7648]
Stops excluded for subroute 111_9: [7647]
Subroute 114_5 ok
Subroute 114_6 ok
Subroute 116_1 ok
Subroute 116_3 ok
Stops excluded for subroute 118_4: [334 345]
Subroute 11_40 ok
Subroute 11_42 ok
Stops excluded for subroute 120_10: [2809 4495 6071]
Stops excluded for subroute 120_11: [ 273  405 6071]
Stops excluded for subroute 120_8: [6004]
Subroute 122_14 ok
Subroute 122_16 ok
Subroute 122_18 ok
Subroute 122_20 ok
Subroute 123_34 ok
Subroute 123_36 ok
Subroute 130_10 ok
Subroute 130_11 ok
Stops excluded for subroute 13_61: [7230]
Stops excluded for subroute 13_63: [7229]
Stops excluded for subroute 13_67: [ 332 7230]
Stops excluded for subroute 13_68: [1999 7230]
Stops excluded for subroute 13_70: [7229 7602]
Subroute 140_19 ok
Subroute 140_21 ok
Subroute 142_12 ok
Stops excluded for subroute 142_13: [ 336 1283 1285 7579 7581]
Stops excluded for subrout

Stops excluded for subroute 83_16: [7132]
Stops excluded for subroute 83_22: [7131]
Stops excluded for subroute 84A_2: [3082 7188]
Stops excluded for subroute 84A_4: [3081 4678 7188 7639 7640]
Stops excluded for subroute 84X_53: [ 303 4307 4308]
Stops excluded for subroute 84X_57: [4952 4956 6137]
Stops excluded for subroute 84X_61: [ 317  791 4307 4308]
Subroute 84_28 ok
Stops excluded for subroute 84_31: [4678]
Subroute 9_5 ok
Stops excluded for subroute 9_7: [2458]
